
An attempt to measure suprise in text using adapters

https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py

In [1]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
import pandas as pd

/media/wassname/SGIronWolf/projects5/bs_writing_detector/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# from peft import LoraConfig, get_peft_model


# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
MAX_LEN = 2000
import frontmatter
from pathlib import Path
from loguru import logger

sample_files = sorted(Path("../samples/").glob('*.md'))
# print(sample_files)
max_chars = 2000
samples = []
for f in sample_files:
    print(f)
    with open(f, "r") as file:
        post = frontmatter.load(file)
        samples.append(dict(content=post.content[:max_chars], f=f, **post.metadata))

df = pd.DataFrame(samples)
df = df[['title', 'f', 'content', 'url', 'novelty', 'date']]
df['date'] = pd.to_datetime(df['date'], utc=True)
df['in_training'] = df.date < '2024-01-01'
df

../samples/2024_anthropic_palintir.md
../samples/2024_arxiv_meh.md
../samples/2024_bob_fanfic.md
../samples/2024_bob_fanfic2.md
../samples/2024_deliberative_alignment.md
../samples/2024_gpt4_fake_paper.md
../samples/2024_gwern_reddit.md
../samples/2024_how_to_focus.md
../samples/2024_lesswrong_slop.md
../samples/2024_news_anthropic.md
../samples/2024_openai_emails.md
../samples/2024_trump_appointment.md
../samples/2025_h5n1_report.md
../samples/2025_lw_human-study-on-ai-spear-phishing-campaigns.md
../samples/2025_lw_parkinson-s-law-and-the-ideology-of-statistics-1.md
../samples/2025_lw_the-intelligence-curse.md
../samples/2025_lw_the-laws-of-large-numbers.md
../samples/2025_lw_what-s-the-short-timeline-plan.md
../samples/lorem_ipsum.md
../samples/politics_is_the_mind_killer.md


,title,f,content,url,novelty,date,in_training
0,Anthropic and Palantir Partner to Bring Claude...,../samples/2024_anthropic_palintir.md,Anthropic and Palantir Technologies Inc. (NYSE...,https://investors.palantir.com/news-details/20...,0.200000,2024-07-11 00:00:00+00:00,False
1,TradingAgents: Multi-Agents LLM Financial Trad...,../samples/2024_arxiv_meh.md,TradingAgents: Multi-Agents LLM Financial Trad...,https://arxiv.org/html/2412.20138v1,0.150000,2024-12-28 00:00:00+00:00,False
2,Flower Crowns and Furry Mishaps by MyPalAI,../samples/2024_bob_fanfic.md,"Bob, April 2222 Delta Eridani\n\n \n\nThere ar...",https://archiveofourown.org/works/61614673/cha...,0.300000,2024-12-24 00:00:00+00:00,False
3,Paradox's Box (Bobiverse) by Mark4man,../samples/2024_bob_fanfic2.md,Chapter 1: Chapter 1: Bill\nChapter Text\n\n“T...,https://archiveofourown.org/works/37201801/cha...,0.400000,2022-02-17 00:00:00+00:00,True
4,Deliberative Alignment: Reasoning Enables Safe...,../samples/2024_deliberative_alignment.md,# Deliberative Alignment: Reasoning Enables Sa...,https://arxiv.org/html/2412.16339v1,0.600000,2024-12-20 00:00:00+00:00,False
5,fake ai hoax paper made up by gpt-4,../samples/2024_gpt4_fake_paper.md,"Title: ""Deconstructing Binaries: Interrogating...",None,0.000000,2024-01-01 00:00:00+00:00,False
6,Hardware Hedging Against Scaling Regime Shifts...,../samples/2024_gwern_reddit.md,Hyperscalers are investing heavily in AMD/Nvid...,https://old.reddit.com/r/mlscaling/comments/1e...,1.000000,2024-08-21 00:00:00+00:00,False
7,How to Focus,../samples/2024_how_to_focus.md,# HOW TO FOCUS\n\n\n\n\n\nANCIENT WISDOM FOR M...,https://github.com/wassname/stoic_markdown/blo...,0.500000,2024-06-01 00:00:00+00:00,False
8,"Deontic Explorations In ""Paying To Talk To Sla...",../samples/2024_lesswrong_slop.md,HER|Jenny|🤔: I want to read a dialogue between...,https://www.lesswrong.com/posts/Rk2o8hjYmjENH8...,0.100000,2024-04-12 00:00:00+00:00,False
9,Amazon-backed Anthropic debuts AI agents that ...,../samples/2024_news_anthropic.md,"![PARIS, FRANCE - MAY 22: Co-founder and CEO o...",https://www.cnbc.com/2024/10/22/anthropic-anno...,0.500000,2025-01-05 05:03:00+00:00,False


## Helpers

In [4]:
from dotenv import load_dotenv

load_dotenv() 

True

In [5]:
import os
from openai import OpenAI
from anycache import anycache

cache_file = "../.anycache.pkl"

@anycache(cachedir=cache_file)
def summize_gpt4(text):
    client = OpenAI()
    content = f"Make a tl;dr of this text in <280 chars.\n\n## Text\n\n{text}\n\n## Instruction\n\nMake a tl;dr of this text in <280 chars. Start with the most important, as extra text will be discarded :\n\ntl;dr:"
    chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="gpt-4",
        )
    r = chat_completion.choices[0].message.content
    return r

@anycache(cachedir=cache_file)
def summarize_gpt4b(text):
    client = OpenAI()
    inst = "We want to compress then reconstruct text. Lets do the compression part. In short hand, record the most usefull information needed to reconstruct the text that can fit in <280 chars. Start with the most important, as extra text will be discarded"
    content = f"{inst}\n\n## Text\n\n{text}\n\n## Instruction\n\n{inst}:\n\ntl;dr:"
    chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="gpt-4",
        )
    # print(content)
    r = chat_completion.choices[0].message.content
    return r

r = summarize_gpt4b(samples[1]["content"])
r

'"TradingAgents" is a multi-agent financial trading model powered by large language models. It features various specialized agents like analysts and traders, mimicking the dynamics of real-world trading firms. The system has shown improved trading performance over baseline models.'

In [6]:
from bs_writing_detector.metrics.ppx import perplexity_compute

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM, AutoConfig, PreTrainedTokenizerBase, PreTrainedTokenizer, GPTQConfig, BitsAndBytesConfig

def load_model(model_name):
    trust_remote_code = True
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    # print(config)
    if config.quantization_config is not None:
        config.quantization_config['disable_exllama'] = True
        if 'use_exllama' in config.quantization_config:
            del config.quantization_config['use_exllama']
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=trust_remote_code, 
                                                 config=config,
                                                 )
    return model, tokenizer


In [8]:
import gc

def clear_mem():
    torch.cuda.empty_cache()
    gc.collect()

## Results

In [9]:
models = [
    # "TheBloke/phi-2-GPTQ",
    # "TheBloke/Llama-2-7B-GPTQ",
    # "TheBloke/Llama-2-13B-GPTQ",
    # "TheBloke/Mistral-7B-v0.1-GPTQ",
    #  "unsloth/Llama-3.2-1B",
     "unsloth/Llama-3.2-1B-bnb-4bit",
]
# model_name = "unsloth/Llama-3.2-1B"
# model_name = "unsloth/Llama-3.2-1B-bnb-4bit"

In [10]:
summaries = {}
data = []
for model_name in models:
    model, tokenizer = load_model(model_name)
    for sample in samples:
        if sample['title'] not in summaries:
            summaries[sample['title']] = summarize_gpt4b(sample['content'])[:600]
        summary = summaries[sample['title']]

        # before 
        s1 = sample['content']
        results = perplexity_compute(data=s1, model=model, tokenizer=tokenizer, device='cuda')
        before = results['mean_perplexity']

        # after 
        s2 = f"""
        High level summary: {summary}

Text:
{sample['content']}
        """
        results = perplexity_compute(data=s2, model=model, tokenizer=tokenizer, device='cuda')
        after = np.array(results['perplexities'])[-len(s1):].mean()

        print(model_name, sample['title'], before, after)
        data.append(dict(before=before, 
                         after=after, 
                         model=model_name, 
                        #  sample=sample['title'],
                        #  in_training=sample['in_training'], 
                         len=len(sample['content']),
                         **sample
                         ),

        )


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method', 'disable_exllama']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
100%|██████████| 1/1 [00:00<00:00,  6.47it/s]


unsloth/Llama-3.2-1B-bnb-4bit Anthropic and Palantir Partner to Bring Claude AI Models to AWS for U.S. Government Intelligence and Defense Operations 15.34375 14.75


100%|██████████| 1/1 [00:00<00:00,  6.66it/s]


unsloth/Llama-3.2-1B-bnb-4bit TradingAgents: Multi-Agents LLM Financial Trading Framework 20.484375 19.359375


100%|██████████| 1/1 [00:00<00:00,  6.47it/s]


...

unsloth/Llama-3.2-1B-bnb-4bit What’s the short timeline plan? 27.9375 28.328125


100%|██████████| 1/1 [00:00<00:00,  6.04it/s]


unsloth/Llama-3.2-1B-bnb-4bit Lorem ipsum 8.0234375 10.2578125


100%|██████████| 1/1 [00:00<00:00,  6.62it/s]

unsloth/Llama-3.2-1B-bnb-4bit politics is the mind-killer 17.65625 20.203125


In [11]:
# results
df2 = pd.DataFrame(data).set_index('title')
df2 = df2.query('in_training == False')
df2["summarizable%"] = (df2["before"] - df2["after"])/df2["before"]

df2

UndefinedVariableError: name 'in_training' is not defined

In [ ]:
df["improvement%"] = (df["before"] - df["after"]) / df["before"] 
df["improvement"] = (df["before"] - df["after"]) 
df["summarizable"] = df["improvement"]  > .0
df['suprising'] = df["before"] > 15
df['BS'] = ~df["summarizable"] | ~df['suprising']
for n,d in df.groupby("model"):
    print(n)
    d = d[[ 'before', 'after', "improvement", "improvement%", 'suprising', 'summarizable',  ]].sort_values("improvement", ascending=False)
    print(d.to_markdown())
    display(d)

In [ ]:
# TODO turn into a single metric, correlate with novelty label